# Proposition Calculus
###### (The Science of Programming, Chapter 2)

Proving the fundamentals of boolean logic propositions.

In [84]:
:set -W
:set -Wno-name-shadowing
:set -Werror
:ext GADTs
:ext DataKinds
:ext TypeOperators
:ext KindSignatures
:ext PolyKinds

import Control.Category
import Data.Proxy
import Data.Type.Bool
import Data.Type.Equality
import Prelude hiding ((.))

data SBool (b :: Bool) where
  STrue :: SBool 'True
  SFalse :: SBool 'False

sNot :: Not a ~ b => SBool a -> SBool b
sNot STrue = SFalse
sNot SFalse = STrue

-- implication
type (a :: Bool) :=> (b :: Bool) = Not a || b

type Tautology a = a :~: 'True

-- Theorems

commAnd :: SBool a -> SBool b -> (a && b) :~: (b && a)
commAnd STrue _ = Refl
commAnd SFalse _ = Refl

commOr :: SBool a -> SBool b -> (a || b) :~: (b || a)
commOr STrue _ = Refl
commOr SFalse _ = Refl

assocAnd :: SBool a -> SBool b -> SBool c
         -> (a && (b && c)) :~: ((a && b) && c)
assocAnd STrue _ _ = Refl
assocAnd SFalse _ _ = Refl

assocOr :: SBool a -> SBool b -> SBool c
        -> (a || (b || c)) :~: ((a || b) || c)
assocOr STrue _ _ = Refl
assocOr SFalse _ _ = Refl

distAnd :: SBool a -> SBool b -> SBool c
        -> (a && (b || c)) :~: ((a && b) || (a && c))
distAnd STrue _ _ = Refl
distAnd SFalse _ _ = Refl

distOr :: SBool a -> SBool b -> SBool c
       -> (a || (b && c)) :~: ((a || b) && (a || c))
distOr STrue _ _ = Refl
distOr SFalse _ _ = Refl

deMorganAnd :: SBool a -> SBool b
            -> Not (a && b) :~: (Not a || Not b)
deMorganAnd STrue _ = Refl
deMorganAnd SFalse _ = Refl

deMorganOr :: SBool a -> SBool b
           -> Not (a || b) :~: (Not a && Not b)
deMorganOr STrue _ = Refl
deMorganOr SFalse _ = Refl

doubleNeg :: SBool a -> Not (Not a) :~: a
doubleNeg STrue = Refl
doubleNeg SFalse = Refl

exclMid :: SBool a -> (Not a || a) :~: 'True
exclMid STrue = Refl
exclMid SFalse = Refl

contradiction :: SBool a -> (a && Not a) :~: 'False
contradiction STrue = Refl
contradiction SFalse = Refl

equality :: SBool a -> SBool b
         -> a :~: b
         -> ((a :=> b) && (b :=> a)) :~: 'True
equality STrue _ Refl = Refl
equality SFalse _ Refl = Refl

conjunctive :: SBool a -> SBool b
            -> (a || b) :~: Not (Not a && Not b)
conjunctive STrue _ = Refl
conjunctive SFalse b =
  case doubleNeg b of
    Refl -> Refl
    
disjunctive :: SBool a -> SBool b
            -> (a && b) :~: Not (Not a || Not b)
disjunctive STrue b =
  case doubleNeg b of
   Refl -> Refl
disjunctive SFalse _ = Refl

-- Lemmas

lemma1 :: SBool a
       -> Not (a && Not a) :~: 'True
lemma1 STrue = Refl
lemma1 SFalse = Refl

lemma2 :: SBool b -> SBool c
       -> ((b && (b :=> c)) :=> c) :~: 'True
lemma2 STrue c = exclMid c
lemma2 SFalse _ = Refl

lemma3 :: SBool a -> SBool b -> SBool c -> SBool d
       -> ((a && b && c) :=> d)
      :~: (Not a || Not b || Not c || d)
lemma3 STrue b c d =
  case deMorganAnd b c of
    Refl -> sym (assocOr (sNot b) (sNot c) d)
lemma3 SFalse _ _ _ = Refl

lemma4 :: SBool b -> SBool c -> SBool d
       -> ((Not (b :=> c) && Not (Not b :=> (c || d))) :=> (Not c :=> d))
     :~: 'True
lemma4 STrue _ _ = Refl
lemma4 SFalse _ _ = Refl

lemma5 :: SBool x -> SBool y
       -> (x || ( y || x) || Not y) :~: 'True
lemma5 STrue _ = Refl
lemma5 SFalse y =
  case commOr (sNot y) y of
    Refl -> exclMid y
    
lemma6 :: SBool x -> SBool y
       -> ((x || y) && (x || Not y)) :~: x
lemma6 STrue _ = Refl
lemma6 SFalse y = contradiction y

lemma7 :: SBool x -> SBool y
       -> (x || y || Not x) :~: 'True
lemma7 STrue _ = Refl
lemma7 SFalse _ = Refl

lemma8 :: SBool x -> SBool y
       -> ((x || y) && (x || Not y) && (Not x || y) && (Not x || Not y)) :~: 'False
lemma8 STrue = contradiction
lemma8 SFalse = contradiction

lemma9 :: SBool x -> SBool y
       -> ((x && y) || (x && Not y) || (Not x && y) || (Not x && Not y)) :~: 'True
lemma9 STrue y =
  case commOr (sNot y) y of
    Refl -> exclMid y
lemma9 SFalse y =
  case commOr (sNot y) y of
    Refl -> exclMid y

lemma10 :: SBool x -> SBool y
        -> ((Not x && y) || x) :~: (x || y)
lemma10 STrue _ = Refl
lemma10 SFalse _ = Refl

lemma11 :: SBool x -> SBool y
        -> (Not x :=> (x && y)) :~: x
lemma11 STrue _ = Refl
lemma11 SFalse _ = Refl

lemma12 :: SBool x
        -> ('True :=> (Not x :=> x)) :~: x
lemma12 STrue = Refl
lemma12 SFalse = Refl

lemma13 :: SBool x -> SBool y
        -> (x :=> (y :=> (x && y))) :~: 'True
lemma13 STrue y = exclMid y
lemma13 SFalse _ = Refl

lemma14 :: SBool x -> SBool y
        -> (Not x :=> (Not x :=> (Not x && y)))
       :~: (x || y)
lemma14 STrue _ = Refl
lemma14 SFalse _ = Refl

lemma15 :: SBool y
        -> (Not y :=> y) :~: y
lemma15 STrue = Refl
lemma15 SFalse = Refl

lemma16 :: SBool y
        -> (Not y :=> Not y) :~: 'True
lemma16 STrue = Refl
lemma16 SFalse = Refl